In [90]:
import numpy as np
import cplex
import random
import time
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [91]:
n_depot, n_vehicle = 10, 3
distance_matrix = pd.read_csv('distance_matrix.csv', names = [str(i) for i in range(n_depot)])
distance_matrix = np.array(distance_matrix)
orders = pd.read_csv('orders.csv')
initial_locations = [4, 10 ,2]
n_order = orders.shape[0] - len(initial_locations)
n_order_all = orders.shape[0]
print(distance_matrix, distance_matrix.shape, type(distance_matrix[0][0]))
print(orders)
print(n_order)

[[ 0 20 10  3 15  8 17 17  8 10]
 [20  0 24 19 12 12  7 13 18 21]
 [10 24  0 13 14 14 24 15  6  3]
 [ 3 19 13  0 16  7 15 18 10 13]
 [15 12 14 16  0  9 15  3  9 10]
 [ 8 12 14  7  9  0 10 12  8 11]
 [17  7 24 15 15 10  0 18 18 21]
 [17 13 15 18  3 12 18  0 10 11]
 [ 8 18  6 10  9  8 18 10  0  3]
 [10 21  3 13 10 11 21 11  3  0]] (10, 10) <class 'numpy.int64'>
    orders  origin depot  destination depot  deadline for pick up
0        1             9                  2                    37
1        2             4                 10                    36
2        3             2                  5                    26
3        4             5                  3                    48
4        5             7                 10                    39
5        6             9                  7                    38
6        7             7                 10                    14
7        8             7                  8                    17
8        9             5                 10 

In [92]:
class Subproblem():
    def __init__(self, n_dim):
        self.n_dim = n_dim
        self.variable_names = ['x' + str(i + 1) for i in range(self.n_dim)]
        self.constraints_rows, self.rhs  = self.set_constraints_byrows()
        self.n_constraints = len(self.constraints_rows)
        self.opt_solution = []
        self.cost_function_iterations = []
    
    def check_dim():
        assert(len(self.obj_coefficents) == len(self.variable_names))
        assert(len(self.obj_coefficents) == self.n_dim)
    
    def get_optvalues(self):
        self.obj_values = self.subproblem_cplex.solution.get_objective_value()
        print ("Solution value:   ",  self.obj_values)
        self.opt_solution = []
        for i, x in enumerate(self.subproblem_cplex.solution.get_values()):
            if i < self.n_dim:
                self.opt_solution.append(x)
                
    def get_solvable_status(self):
        status = self.subproblem_cplex.solution.get_status()
        if status == self.subproblem_cplex.solution.status.unbounded:
            print("Model is unbounded")
        if status == self.subproblem_cplex.solution.status.infeasible:
            print("Model is infeasible")
        if status == self.subproblem_cplex.solution.status.infeasible_or_unbounded:
            print("Model is infeasible or unbounded")
    
    def set_method(self, method):
        alg = self.subproblem_cplex.parameters.lpmethod.values

        if method == "o":  
            self.subproblem_cplex.parameters.lpmethod.set(alg.auto)
        elif method == "p":
            self.subproblem_cplex.parameters.lpmethod.set(alg.primal)
        elif method == "d":
            self.subproblem_cplex.parameters.lpmethod.set(alg.dual)
        elif method == "b":
            self.subproblem_cplex.parameters.lpmethod.set(alg.barrier)
            self.subproblem_cplex.parameters.barrier.crossover.set(self.Subproblem_LP_TU.parameters.parameters.barrier.crossover.values.none)
        elif method == "h":
            self.subproblem_cplex.parameters.lpmethod.set(alg.barrier)
        elif method == "s":
            self.subproblem_cplex.parameters.lpmethod.set(alg.sifting)
        elif method == "c":
            self.subproblem_cplex.parameters.lpmethod.set(alg.concurrent)
        else:
            raise ValueError('method must be one of "o", "p", "d", "b", "h", "s" or "c"')
    
    def solve(self, method = 'o'):
        self.subproblem_cplex = cplex.Cplex()
        self.subproblem_cplex.objective.set_sense(self.subproblem_cplex.objective.sense.maximize)
        lb, ub = self.set_lb_ub()
        self.obj_coefficents = self.set_obj()
        self.subproblem_cplex.variables.add(obj = self.obj_coefficents, lb = lb, ub = ub, names = self.variable_names)   
        
        self.senses = self.set_senses()
        self.rhs = self.set_rhs()
        row_names = ['c'+ str(i + 1) for i in range(self.n_constraints)]

        self.subproblem_cplex.linear_constraints.add(lin_expr = self.constraints_rows, senses = self.senses, rhs = self.rhs, names = row_names)
        self.set_method(method)
        self.subproblem_cplex.solve()
        self.get_solvable_status()
        self.get_optvalues()
        

In [93]:
class Subproblem_x(Subproblem):
    def __init__(self, n_dim):
        Subproblem.__init__(self, n_dim)
        self.big_m = 570.0
        
    def set_constraints_byrows(self):
        order_to_vehicle = self.compute_constraints_order_to_vehicle()
        initial_locations_to_vehicle = self.compute_constraints_initial_locations_to_vehicle()
        rhs = self.set_rhs()
        return order_to_vehicle + initial_locations_to_vehicle, rhs
       
    def compute_constraints_order_to_vehicle(self):
        order_to_vehicle = []
        for i in range(n_order):
            order_to_vehicle.append([self.variable_names[i: self.n_dim : n_order_all], [1] * n_vehicle])
        return order_to_vehicle
    
    def compute_constraints_initial_locations_to_vehicle(self):
        initial_locations_to_vehicle = []
        n_initial_location = n_order_all - n_order
        for i in range(n_initial_location):
            for j in range(n_initial_location):
                initial_locations_to_vehicle.append([[self.variable_names[i * n_order_all + j + n_order]], [1]])

        return initial_locations_to_vehicle
    
    def set_rhs(self):
        rhs = [1] * n_order
        rhs = rhs + [1, 0, 0, 0 ,1, 0, 0, 0, 1]
        return rhs
    
    def set_senses(self):
        senses = 'L'* n_order + 'E' * (n_order_all - n_order) * (n_order_all - n_order)
        return senses
    
    def set_obj(self):
        obj_coefficents = self.compute_obj_f1() + self.compute_obj_f2()
        return obj_coefficents
    
    def compute_obj_f1(self):
        self.orders_time = self.compute_orders_time()
        obj_coefficents_f1 = self.orders_time * n_vehicle
        return np.array(obj_coefficents_f1)
    
    def compute_orders_time(self):
        orders_time = []
        for i in range(n_order_all):
            orders_time.append(int(distance_matrix[orders['origin depot'][i] - 1][orders['destination depot'][i] - 1]))
        return orders_time
    
    def compute_obj_f2(self):
        obj_coefficents_f2 = np.zeros((n_vehicle, n_order_all))
        for i in range(n_order_all):
            for j in range(n_order_all):
                for k in range(n_vehicle):
                    if i!=j:
                        obj_coefficents_f2[k][i] -= lamda[i][j][k]
                        obj_coefficents_f2[k][j] -= lamda[i][j][k]
                        
        obj_coefficents_f2 = self.big_m * obj_coefficents_f2.reshape(self.n_dim)
        return obj_coefficents_f2
    
    def set_lb_ub(self):
        ub = [1] * self.n_dim
        lb = [0] * self.n_dim
        return lb, ub
                          
    def write_model(self):
        self.subproblem_cplex.write('Subproblem_x.lp')       
        
    def output_results(self):
        path = os.getcwd()
        path = path + '\\results\\'
        with open(path + 'Subproblem_x_result.txt', 'w') as f:
            for i in range(n_vehicle):
                f.write(str(self.opt_solution[i * n_order_all: (i + 1) * n_order_all]))
                f.write('\n')

In [94]:
n_dim = n_order_all * n_vehicle
subproblem_x = Subproblem_x(n_dim)
print(subproblem_x.set_constraints_byrows())

([[['x1', 'x16', 'x31'], [1, 1, 1]], [['x2', 'x17', 'x32'], [1, 1, 1]], [['x3', 'x18', 'x33'], [1, 1, 1]], [['x4', 'x19', 'x34'], [1, 1, 1]], [['x5', 'x20', 'x35'], [1, 1, 1]], [['x6', 'x21', 'x36'], [1, 1, 1]], [['x7', 'x22', 'x37'], [1, 1, 1]], [['x8', 'x23', 'x38'], [1, 1, 1]], [['x9', 'x24', 'x39'], [1, 1, 1]], [['x10', 'x25', 'x40'], [1, 1, 1]], [['x11', 'x26', 'x41'], [1, 1, 1]], [['x12', 'x27', 'x42'], [1, 1, 1]], [['x13'], [1]], [['x14'], [1]], [['x15'], [1]], [['x28'], [1]], [['x29'], [1]], [['x30'], [1]], [['x43'], [1]], [['x44'], [1]], [['x45'], [1]]], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1])


In [95]:
print(subproblem_x.compute_orders_time())

[18, 13, 12, 14, 21, 18, 21, 18, 10, 10, 8, 13, 0, 0, 0]


In [96]:
print(subproblem_x.compute_obj_f1())

[18 13 12 14 21 18 21 18 10 10  8 13  0  0  0 18 13 12 14 21 18 21 18 10
 10  8 13  0  0  0 18 13 12 14 21 18 21 18 10 10  8 13  0  0  0]


In [97]:
lamda = - np.ones((n_order_all, n_order_all, n_vehicle))
print(subproblem_x.compute_obj_f2())

[15960. 15960. 15960. 15960. 15960. 15960. 15960. 15960. 15960. 15960.
 15960. 15960. 15960. 15960. 15960. 15960. 15960. 15960. 15960. 15960.
 15960. 15960. 15960. 15960. 15960. 15960. 15960. 15960. 15960. 15960.
 15960. 15960. 15960. 15960. 15960. 15960. 15960. 15960. 15960. 15960.
 15960. 15960. 15960. 15960. 15960.]


In [98]:
print(subproblem_x.set_obj())

[15978. 15973. 15972. 15974. 15981. 15978. 15981. 15978. 15970. 15970.
 15968. 15973. 15960. 15960. 15960. 15978. 15973. 15972. 15974. 15981.
 15978. 15981. 15978. 15970. 15970. 15968. 15973. 15960. 15960. 15960.
 15978. 15973. 15972. 15974. 15981. 15978. 15981. 15978. 15970. 15970.
 15968. 15973. 15960. 15960. 15960.]


In [99]:
subproblem_x.solve()
subproblem_x.write_model()
subproblem_x.output_results()

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 21 rows and 45 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)
Solution value:    239576.0


In [100]:
class Subproblem_y():
    def __init__(self, n_dim):
        self.n_dim = n_dim
        self.big_m = 570.0
        self.obj_values = 0
        self.opt_solution = self.set_init_opt_solution()
        self.miu = self.compute_miu()
    
    def compute_miu(self):
        miu = np.zeros((n_order, n_order))
        for i in range(n_order):
            for j in range(n_order):
                miu[i][j] = lamda[i][j][:].sum()
        return miu
    
    def set_init_opt_solution(self):
        opt_solution = np.zeros((n_order_all, n_order_all))
        opt_solution[n_order:n_order_all, 0:n_order] = np.ones((n_vehicle, 1))
        return opt_solution
    
    def solve(self):
        for i in range(n_order):
            for j in range(n_order):
                if (i < j):
                    if (self.miu[i][j] >= self.miu[j][i]):
                        self.opt_solution[i][j] = 1 
                    else:
                        self.opt_solution[j][i] = 1 
                        
    def compute_obj_values(self):    
        self.obj_values = 0
        for k in range(n_vehicle):
            for i in range(n_order_all):
                for j in range(n_order_all):
                    if i!=j:
                        self.obj_values -= lamda[i][j][k] * self.opt_solution[i][j]
        self.obj_values = self.obj_values * self.big_m
        return self.obj_values    
    
    def output_results(self):
        path = os.getcwd()
        path = path + '\\results\\'
        with open(path + 'Subproblem_y_result.txt', 'w') as f:
            for i in range(n_order_all):
                f.write(str(self.opt_solution[i:n_order_all: (i + 1)*n_order_all]))
                f.write('\n')

In [101]:
subproblem_y = Subproblem_y(((n_order + n_vehicle) * n_order))
print(subproblem_y.opt_solution)
print(subproblem_y.compute_miu())
subproblem_y.solve()
print(subproblem_y.opt_solution)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0.]]
[[-3. -3. -3. -3. -3. -3. -3. -3. -3. -3. -3. -3.]
 [-3. -3. -3. -3. -3. -3. -3. -3. -3. -3. -3. -3.]
 [-3. -3. -3. -3. -3. -3. -3. -3. -3. -3. -3. -3.]
 [-3. -3. -3. -3. -3. -3. -3. -3. -3. -3. -3. -3.]
 [-3. -3. -3. -3. -3. -3. -3. -3. -3. -3. -3. -3.]
 [-3. -3. -3. -3. -3. -3

In [102]:
print(subproblem_y.compute_obj_values())
subproblem_y.output_results()

174420.0


In [103]:
class Subproblem_time(Subproblem):
    def __init__(self, n_dim):
        Subproblem.__init__(self, n_dim)
        
    def set_constraints_byrows(self):
        order_to_vehicle = self.compute_constraints_origin_to_destination()
        rhs = self.set_rhs()
        return order_to_vehicle, rhs
       
    def compute_constraints_origin_to_destination(self):
        origin_to_destination = []
        for i in range(n_order_all):
            origin_to_destination.append([self.variable_names[i: self.n_dim: n_order_all], [-1, 1]])
        return origin_to_destination

    
    def set_rhs(self):
        orders_time = self.compute_orders_time()
        rhs = orders_time
        return rhs
    
    def compute_orders_time(self):
        orders_time = []
        for i in range(n_order_all):
            orders_time.append(int(distance_matrix[orders['origin depot'][i] - 1][orders['destination depot'][i] - 1]))
        return orders_time
    
    def set_senses(self):
        senses = 'E'* self.n_constraints
        return senses
    
    def set_obj(self):
        obj_coefficents = self.compute_obj_f5()
        return obj_coefficents
    
    def compute_obj_f5(self):
        obj_coefficents_f5 = np.zeros(self.n_dim)
        for i in range(n_order):
            for j in range(n_order):
                for k in range(n_vehicle):
                    if i!=j:
                        obj_coefficents_f5[i] += lamda[i][j][k]
                        obj_coefficents_f5[j] -= lamda[i][j][k]
                        
        return obj_coefficents_f5
    
    def set_lb_ub(self):
        ub = orders['deadline for pick up'].tolist() + [cplex.infinity] * n_order_all
        lb = [0] * self.n_dim
        return lb, ub
                        
    def write_model(self):
        self.subproblem_cplex.write('Subproblem_time.lp')       
    
    def output_results(self):
        path = os.getcwd()
        path = path + '\\results\\'
        with open(path + 'Subproblem_time_result.txt', 'w') as f:
            f.write(str(self.opt_solution))
            f.write('\n')

In [104]:
n_dim = 2 * n_order_all
subproblem_time = Subproblem_time(n_dim)
print(subproblem_time.set_constraints_byrows())
print(subproblem_time.set_rhs())
print(subproblem_time.set_lb_ub())

([[['x1', 'x16'], [-1, 1]], [['x2', 'x17'], [-1, 1]], [['x3', 'x18'], [-1, 1]], [['x4', 'x19'], [-1, 1]], [['x5', 'x20'], [-1, 1]], [['x6', 'x21'], [-1, 1]], [['x7', 'x22'], [-1, 1]], [['x8', 'x23'], [-1, 1]], [['x9', 'x24'], [-1, 1]], [['x10', 'x25'], [-1, 1]], [['x11', 'x26'], [-1, 1]], [['x12', 'x27'], [-1, 1]], [['x13', 'x28'], [-1, 1]], [['x14', 'x29'], [-1, 1]], [['x15', 'x30'], [-1, 1]]], [18, 13, 12, 14, 21, 18, 21, 18, 10, 10, 8, 13, 0, 0, 0])
[18, 13, 12, 14, 21, 18, 21, 18, 10, 10, 8, 13, 0, 0, 0]
([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [37, 36, 26, 48, 39, 38, 14, 17, 17, 12, 38, 46, 0, 0, 0, 1e+20, 1e+20, 1e+20, 1e+20, 1e+20, 1e+20, 1e+20, 1e+20, 1e+20, 1e+20, 1e+20, 1e+20, 1e+20, 1e+20, 1e+20])


In [105]:
lamda[10][0][1] = -20
print(subproblem_time.set_obj())

[ 19.   0.   0.   0.   0.   0.   0.   0.   0.   0. -19.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.]


In [106]:
subproblem_time.solve()
subproblem_time.write_model()
subproblem_time.output_results()

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 15 rows and 30 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)
Solution value:    703.0


In [107]:
class Dual_Problem():
    def __init__(self):
        self.lamda = - np.ones((n_order_all, n_order_all, n_vehicle))
        self.m, self.r, self.alpha = 5, 0.01, 1.0
        self.step_init, self.step, self.iteration_time = 0.0, 0.0, 0
        self.subgradients = - np.zeros((n_order_all, n_order_all, n_vehicle))
        self.obj_values_iterations = []
        self.subgradients_norm_iterations = []
        self.step_iterations = []
        self.obj_values = 0
        self.big_m = 570
    
    def init_subproblem_opt_solution(self, x, y, t):
        self.x, self.y, self.t = x, y, t
        
    def compute_orders_transfer_time(self):
        self.orders_transfer_time = np.zeros((n_order_all, n_order_all))
        for i in range(n_order_all):
            for j in range(n_order_all):
                 self.orders_transfer_time[i][j] = distance_matrix[orders['destination depot'][i] - 1][orders['origin depot'][j] - 1]
                
    def compute_subgradients(self):
        for i in range(n_order_all):
            for j in range(n_order_all):
                for k in range(n_vehicle):
                    if i!=j:
                        self.subgradients[i][j][k] = (self.t[i] + self.orders_transfer_time[i][j] -  self.t[j] + (3 - self.y[i][j] -  self.x[k*n_order_all+i] -  self.x[k*n_order_all+j])* self.big_m)
                    
    def compute_stepsize(self):
        self.step = 0.1
        return self.step
    
    def update_lamd(self):
        self.lamda = self.lamda - self.step * self.subgradients
        self.lamda = np.mimimum(self.lamda, 0)
        self.iteration_time += 1
        return self.lamd
    
    def compute_relaxed_problem_obj_values(self, orders_time):
        self.obj_values = 0
        for i in range(n_order_all):
            for j in range(n_order_all):
                for k in range(n_vehicle):
                    if i!=j:
                        self.obj_values += self.lamda[i][j][k] * (self.t[i] + self.orders_transfer_time[i][j] -  self.t[j] + (3 - self.y[i][j] -  self.x[k*n_order_all+i] -  self.x[k*n_order_all+j])* self.big_m)
        
        for j in range(n_order):
            for i in range(n_vehicle):
                self.obj_values += self.x[i*n_order_all + j] * orders_time[j]


In [108]:
dual_problem = Dual_Problem()
dual_problem.compute_orders_transfer_time()
print(dual_problem.orders_transfer_time)

[[18. 19.  0. 12.  7. 18.  7.  7. 12. 19. 20. 19. 19. 21.  0.]
 [ 3. 13. 21. 10. 21.  3. 21. 21. 10. 13. 10. 13. 13.  0. 21.]
 [ 9. 16. 12.  0. 15.  9. 15. 15.  0. 16. 15. 16. 16. 10. 12.]
 [ 6. 13. 24. 14. 24.  6. 24. 24. 14. 13. 10. 13. 13.  3. 24.]
 [ 3. 13. 21. 10. 21.  3. 21. 21. 10. 13. 10. 13. 13.  0. 21.]
 [18. 15.  7. 15.  0. 18.  0.  0. 15. 15. 17. 15. 15. 21.  7.]
 [ 3. 13. 21. 10. 21.  3. 21. 21. 10. 13. 10. 13. 13.  0. 21.]
 [10. 18. 13.  3. 18. 10. 18. 18.  3. 18. 17. 18. 18. 11. 13.]
 [ 3. 13. 21. 10. 21.  3. 21. 21. 10. 13. 10. 13. 13.  0. 21.]
 [ 0. 10. 18.  9. 18.  0. 18. 18.  9. 10.  8. 10. 10.  3. 18.]
 [ 8.  7. 12.  9. 10.  8. 10. 10.  9.  7.  8.  7.  7. 11. 12.]
 [ 6. 13. 24. 14. 24.  6. 24. 24. 14. 13. 10. 13. 13.  3. 24.]
 [10.  0. 19. 16. 15. 10. 15. 15. 16.  0.  3.  0.  0. 13. 19.]
 [ 3. 13. 21. 10. 21.  3. 21. 21. 10. 13. 10. 13. 13.  0. 21.]
 [18. 19.  0. 12.  7. 18.  7.  7. 12. 19. 20. 19. 19. 21.  0.]]


In [109]:
dual_problem.init_subproblem_opt_solution(subproblem_x.opt_solution, subproblem_y.opt_solution, subproblem_time.opt_solution)
dual_problem.compute_subgradients()
print(dual_problem.subgradients)

[[[  -0.   -0.   -0.]
  [  56. 1196. 1196.]
  [  37. 1177. 1177.]
  [  49. 1189. 1189.]
  [  44. 1184. 1184.]
  [  55. 1195. 1195.]
  [  44. 1184. 1184.]
  [  44. 1184. 1184.]
  [  49. 1189. 1189.]
  [  56. 1196. 1196.]
  [  57. 1197. 1197.]
  [  56. 1196. 1196.]
  [ 626. 1766. 1766.]
  [1198. 1198. 1768.]
  [1177. 1747. 1177.]]

 [[ 536. 1676. 1676.]
  [  -0.   -0.   -0.]
  [  21. 1161. 1161.]
  [  10. 1150. 1150.]
  [  21. 1161. 1161.]
  [   3. 1143. 1143.]
  [  21. 1161. 1161.]
  [  21. 1161. 1161.]
  [  10. 1150. 1150.]
  [  13. 1153. 1153.]
  [  10. 1150. 1150.]
  [  13. 1153. 1153.]
  [ 583. 1723. 1723.]
  [1140. 1140. 1710.]
  [1161. 1731. 1161.]]

 [[ 542. 1682. 1682.]
  [ 586. 1726. 1726.]
  [  -0.   -0.   -0.]
  [   0. 1140. 1140.]
  [  15. 1155. 1155.]
  [   9. 1149. 1149.]
  [  15. 1155. 1155.]
  [  15. 1155. 1155.]
  [   0. 1140. 1140.]
  [  16. 1156. 1156.]
  [  15. 1155. 1155.]
  [  16. 1156. 1156.]
  [ 586. 1726. 1726.]
  [1150. 1150. 1720.]
  [1152. 1722. 1152.]]

 [[ 

In [111]:
dual_problem.compute_relaxed_problem_obj_values(subproblem_x.orders_time)
print(dual_problem.obj_values)

-671224.0
